## Benchmarking Classification Algorithms: A Marketing Use Case

**Overview**:  
In this project, we benchmark the performance of four popular classification algorithms: **K-Nearest Neighbors (KNN)**, **Logistic Regression**, **Decision Trees**, and **Support Vector Machines (SVM)**. The evaluation is conducted using a dataset derived from **telephone marketing campaigns for bank products**. Our comparison focuses on key metrics such as **training time**, **training accuracy**, and **testing accuracy**.

---

### 1. Data

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



As per the paper (Section:  **Materials and Methods** ), this data is based on 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts.

 

In [47]:
import pandas as pd
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')
df.head()


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### 2. Features Interpretation


In [48]:
### Features Interpretation

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Features Interpretation (Tabular Format)

| **Feature**           | **Description** |
|------------------------|-----------------|
| **1 - age**            | Numeric — Age of the client |
| **2 - job**            | Categorical — Type of job ('admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown') |
| **3 - marital**        | Categorical — Marital status ('divorced','married','single','unknown') |
| **4 - education**      | Categorical — Education level ('basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown') |
| **5 - default**        | Categorical — Has credit in default? ('no','yes','unknown') |
| **6 - housing**        | Categorical — Has housing loan? ('no','yes','unknown') |
| **7 - loan**           | Categorical — Has personal loan? ('no','yes','unknown') |
| **8 - contact**        | Categorical — Contact communication type ('cellular','telephone') |
| **9 - month**          | Categorical — Last contact month ('jan','feb','mar',...,'dec') |
| **10 - day_of_week**   | Categorical — Last contact day of the week ('mon','tue','wed','thu','fri') |
| **11 - duration**      | Numeric — Duration of last contact (in seconds). *Note: Strongly affects target and should only be used for benchmark purposes.* |
| **12 - campaign**      | Numeric — Number of contacts in this campaign |
| **13 - pdays**         | Numeric — Days since last contact (999 = not previously contacted) |
| **14 - previous**      | Numeric — Number of contacts before this campaign |
| **15 - poutcome**      | Categorical — Outcome of previous campaign ('failure','nonexistent','success') |
| **16 - emp.var.rate**  | Numeric — Employment variation rate (quarterly) |
| **17 - cons.price.idx**| Numeric — Consumer price index (monthly) |
| **18 - cons.conf.idx** | Numeric — Consumer confidence index (monthly) |
| **19 - euribor3m**     | Numeric — Euribor 3 month rate (daily) |
| **20 - nr.employed**   | Numeric — Number of employees (quarterly) |
| **21 - y**             | Binary (Target) — Client subscribed to term deposit? ('yes','no') |


In [49]:
# Check for null values
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

### 3. Task Goal

#### Business Objective
The primary goal is to **predict whether a bank client will subscribe to a term deposit** based on a variety of personal, contact, and socio-economic features.

This classification task supports the bank's **marketing efforts** by helping identify potential clients who are more likely to respond positively to a term deposit offer. By targeting these individuals more effectively, the bank can:

- Improve the success rate of marketing campaigns  
- Optimize resource allocation (e.g., time spent by agents, call costs)  
- Increase overall return on investment (ROI)  
- Enhance customer engagement through personalized offers  

#### Machine Learning Task
This is a **binary classification** problem where the target variable is:

- `y`: *Has the client subscribed to a term deposit?*
  - `yes` → client subscribed
  - `no` → client did not subscribe

The model will learn patterns from historical data to predict this outcome for future or unseen clients.


### 4. Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [50]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
# print (df['job'].unique())
nominal_categorical_columns = [
    'job',
    'marital',
    'default',
    'housing',
    'loan',
    'contact',
    'month',
    'day_of_week',
    'poutcome'
]
le = LabelEncoder ()
for column in nominal_categorical_columns:
    df[column] = le.fit_transform(df[column])

ordinal_categorical_columns = ['education']


education_order = {
    'university.degree': 6,
    'professional.course': 5,
    'high.school': 4,
    'basic.9y': 3,
    'basic.6y': 2,
    'basic.4y': 1,
    'unknown': 0,
    'illiterate': -1
}
df['education'] = df['education'].map(education_order)

df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,1,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
1,57,7,1,4,1,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
2,37,7,1,4,0,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
3,40,0,1,2,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
4,56,7,1,4,0,0,2,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,5,1,5,0,2,0,0,7,0,...,1,999,0,1,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,1,1,5,0,0,0,0,7,0,...,1,999,0,1,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,5,1,6,0,2,0,0,7,0,...,2,999,0,1,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,9,1,5,0,0,0,0,7,0,...,1,999,0,1,-1.1,94.767,-50.8,1.028,4963.6,yes


In [51]:
y = df['y'].map({'yes': 1, 'no': 0})
X = df.drop(columns=['y'])
standard_scaler = StandardScaler()
X = standard_scaler.fit_transform(X)

### 5. Train/Test Split

With your data prepared, split it into a train and test set.

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### 6. A Baseline Model

Creating baseline model of Logistic regression.

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)
train_time = time.time() - start_time

y_pred = logistic_regression.predict(X_test)
train_accuracy_score = accuracy_score(y_train, y_train)
test_accuracy_score = accuracy_score(y_test, y_pred)
print (f"{train_time:0.2f} Seconds")
train_accuracy_score, test_accuracy_score

0.21 Seconds


(1.0, 0.9084729303228939)

### 7. Comparing models using Grid Search Cross Validation

In [54]:
model_comarison_metrics = [['model_name', 'Train_time', 'train_accuracy', 'test_accuracy']]

In [55]:
#K Nearest Neighbor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier()
grid_params= {'n_neighbors':[3, 5 ,7]}
grid_search_cv_knn = GridSearchCV(estimator = knn, param_grid=grid_params)


start_time = time.time()
grid_search_cv_knn.fit(X_train, y_train)
train_time = time.time() - start_time

y_pred = grid_search_cv_knn.predict(X_test)

train_accuracy_score = accuracy_score(y_train, y_train)
test_accuracy_score = accuracy_score(y_test, y_pred)
print (f"{train_time:0.2f} Seconds")
train_accuracy_score, test_accuracy_score

model_comarison_metrics.append(['KNN', train_time, train_accuracy_score, test_accuracy_score ])
model_comarison_metrics

12.19 Seconds


[['model_name', 'Train_time', 'train_accuracy', 'test_accuracy'],
 ['KNN', 12.188496589660645, 1.0, 0.9000566480537348]]

In [56]:
# Logistic Regression
grid_params = {'C': [0.1, 1, 10]}

logistic_regression = LogisticRegression(max_iter=1000)
grid_search_cv_lr = GridSearchCV(estimator = logistic_regression, param_grid=grid_params)


start_time = time.time()
grid_search_cv_lr.fit(X_train, y_train)
train_time = time.time() - start_time

y_pred = grid_search_cv_lr.predict(X_test)

train_accuracy_score = accuracy_score(y_train, y_train)
test_accuracy_score = accuracy_score(y_test, y_pred)
print (f"{train_time:0.2f} Seconds")
train_accuracy_score, test_accuracy_score

model_comarison_metrics.append(['Logistic Regression', train_time, train_accuracy_score, test_accuracy_score ])
model_comarison_metrics


2.23 Seconds


[['model_name', 'Train_time', 'train_accuracy', 'test_accuracy'],
 ['KNN', 12.188496589660645, 1.0, 0.9000566480537348],
 ['Logistic Regression', 2.2317395210266113, 1.0, 0.9094440398154892]]

In [57]:
#Decision Trees
from sklearn.tree import DecisionTreeClassifier

grid_params = {'max_depth': [5, 10, 15]}


decision_tree = DecisionTreeClassifier()
grid_search_cv_tree = GridSearchCV(estimator = decision_tree, param_grid=grid_params)


start_time = time.time()
grid_search_cv_tree.fit(X_train, y_train)
train_time = time.time() - start_time

y_pred = grid_search_cv_tree.predict(X_test)

train_accuracy_score = accuracy_score(y_train, y_train)
test_accuracy_score = accuracy_score(y_test, y_pred)
print (f"{train_time:0.2f} Seconds")
train_accuracy_score, test_accuracy_score

model_comarison_metrics.append(['Decision Trees', train_time, train_accuracy_score, test_accuracy_score ])
model_comarison_metrics

2.72 Seconds


[['model_name', 'Train_time', 'train_accuracy', 'test_accuracy'],
 ['KNN', 12.188496589660645, 1.0, 0.9000566480537348],
 ['Logistic Regression', 2.2317395210266113, 1.0, 0.9094440398154892],
 ['Decision Trees', 2.7186224460601807, 1.0, 0.9129238488306223]]

In [58]:
# Support Vector Machines
from sklearn.svm import SVC

grid_params = {'C': [0.1, 1, 10]}

support_vector_classifier = SVC(kernel='linear', degree=2)
grid_search_cv = GridSearchCV(estimator = support_vector_classifier, param_grid=grid_params)


start_time = time.time()
grid_search_cv.fit(X_train, y_train)
train_time = time.time() - start_time

y_pred = grid_search_cv.predict(X_test)

train_accuracy_score = accuracy_score(y_train, y_train)
test_accuracy_score = accuracy_score(y_test, y_pred)
print (f"{train_time:0.2f} Seconds")
train_accuracy_score, test_accuracy_score

1016.92 Seconds


(1.0, 0.8990046127700898)

In [59]:

model_comarison_metrics.append(['Support Vector Machine', train_time, train_accuracy_score, test_accuracy_score ])
model_comarison_metrics

[['model_name', 'Train_time', 'train_accuracy', 'test_accuracy'],
 ['KNN', 12.188496589660645, 1.0, 0.9000566480537348],
 ['Logistic Regression', 2.2317395210266113, 1.0, 0.9094440398154892],
 ['Decision Trees', 2.7186224460601807, 1.0, 0.9129238488306223],
 ['Support Vector Machine', 1016.919709444046, 1.0, 0.8990046127700898]]

In [66]:
model_comparison_df = pd.DataFrame(model_comarison_metrics[1:], columns=model_comarison_metrics[0])
model_comparison_df

,model_name,Train_time,train_accuracy,test_accuracy
0,KNN,12.188497,1.0,0.900057
1,Logistic Regression,2.231740,1.0,0.909444
2,Decision Trees,2.718622,1.0,0.912924
3,Support Vector Machine,1016.919709,1.0,0.899005


In [61]:
#Best Parameters - SVM
grid_search_cv.best_estimator_, grid_search_cv.best_params_


(SVC(C=0.1, degree=2, kernel='linear'), {'C': 0.1})

In [62]:
#Best Parameters - K Nearest Neighbor
grid_search_cv_knn.best_estimator_, grid_search_cv_knn.best_params_

(KNeighborsClassifier(n_neighbors=7), {'n_neighbors': 7})

In [63]:
#Best Parameters - Logistic Regression
grid_search_cv_lr.best_estimator_, grid_search_cv_lr.best_params_

(LogisticRegression(C=0.1, max_iter=1000), {'C': 0.1})

In [64]:
#Best Parameters - Decision Tree
grid_search_cv_tree.best_estimator_, grid_search_cv_tree.best_params_

(DecisionTreeClassifier(max_depth=5), {'max_depth': 5})

### 8. Conclusion

All four classifiers—**K-Nearest Neighbors (KNN)**, **Logistic Regression**, **Decision Trees**, and **Support Vector Machine (SVM)**—achieved perfect training accuracy, indicating that each model fit the training data exceptionally well. However, differences in test accuracy and training time reveal key trade-offs:

- **Decision Trees** delivered the **highest test accuracy (91.29%)** with a **relatively low training time (2.72 seconds)**, making it the most balanced and efficient model in this comparison.
- **Logistic Regression** followed closely with **90.94% test accuracy** and the **fastest training time (2.23 seconds)**, offering excellent performance with minimal computational cost.
- **KNN**, while also quick to train, had a slightly lower test accuracy (**90.01%**), suggesting it may be more sensitive to data complexity.
- **Support Vector Machine (SVM)** had the **longest training time (over 1000 seconds)** and the **lowest test accuracy (89.90%)**, indicating it is less efficient and may be overfitting or less suited to this dataset.

---

### 9. Summary

**Decision Trees** and **Logistic Regression** stand out as the most effective classifiers for this task, offering a strong balance between accuracy and computational efficiency.
